<a href="https://colab.research.google.com/github/Bayrem-ben/Bayrem-ben.github.io/blob/master/Bayrem_of_fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
df_train = pd.read_table(train_file_path,sep='\t', names=['spam', 'text'])
df_test = pd.read_table(test_file_path,sep='\t', names=['spam', 'text']) 

df_train['spam'] = df_train['spam'].replace({'ham': 0, 'spam': 1})
df_test['spam'] = df_test['spam'].replace({'ham': 0, 'spam': 1})

In [ ]:
import nltk
from nltk.corpus import stopwords
import string

nltk.download("stopwords")
def process_text(text):
  nopunc = [char for char in text if char not in string.punctuation]
  nopunc = ''.join(nopunc)

  clean_words = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
  return clean_words
  
df_train['text'] = df_train['text'].apply(process_text)
df_test['text'] = df_test['text'].apply(process_text)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

train_data = CountVectorizer(analyzer=process_text).fit_transform(df_train['text'])
test_data = CountVectorizer(analyzer=process_text).fit_transform(df_test['text'])
train_label = df_train['spam']
test_label = df_test['spam']
#another methode predict msg spam
"""
classifier = MultinomialNB().fit(train_data, train_label)
pred = classifier.predict(train_data)
diff = classification_report(train_label, pred)
print(diff)
print('Confustion matrix : \n', confusion_matrix(train_label, pred))
print('Accuracy : ', accuracy_score(train_label, pred))

classifier = MultinomialNB().fit(test_data, test_label)
pred = classifier.predict(test_data)
diff = classification_report(test_label, pred)
print(diff)
print('Confustion matrix : \n', confusion_matrix(test_label, pred))
print('Accuracy : ', accuracy_score(test_label, pred))
"""

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Masking, Embedding

model = Sequential()
# Embedding layer
model.add(Embedding(88584, 32))
# Masking layer for pre-trained embeddings
model.add(Masking(mask_value=0.0))
# Recurrent layer
model.add(LSTM(64, return_sequences=False, dropout=0.1, recurrent_dropout=0.1))
# Fully connected layer
model.add(Dense(64, activation='relu'))
# Dropout for regularization
model.add(Dropout(0.5))
# Output layer
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()

tokenizer=tf.keras.preprocessing.text.Tokenizer(
    num_words=None,
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
    lower=True, split=' ', char_level=False)
sequences = tokenizer.texts_to_sequences(train_data)
seq = text_to_word_sequence(sequences, filters=self.filters, 
                            lower=self.lower, split=self.split)

embedding_matrix = np.zeros((len(vectors), vectors.shape[0]))
x = sequences_to_matrix(sequences, mode='binary')
vec = sequences_to_texts_generator(sequences)
train = texts_to_matrix(vec, mode='binary')
"""
vectors = train_label
words = train_data
# Create lookup of words to vectors
word_lookup = {word: vector for word, vector in zip(words, vectors)}
# New matrix to hold word embeddings
embedding_matrix = np.zeros((len(vectors), vectors.shape[0]))
for i, word in enumerate(word_lookup.keys()):
    # Look up the word embedding
    vector = word_lookup.get(word, None)
    # Record in matrix
    if vector is not None:
        embedding_matrix[i + 1, :] = vector
"""
history = model.fit(train_data, train_label, epochs=10, validation_split = 0.2, verbose=1)

In [ ]:
  """
  titles = list(piv.index.values)
  distances, indices = model.kneighbors(piv.loc[book].values.reshape(1, -1), len(titles), True)
  recommended_books = [book, sum([[[piv.index[indices.flatten()[i]], distances.flatten()[i]]] for i in range(5, 0, -1)], [])]
  """
  #imdb
word_index = imdb.get_word_index()
def encoded_text(text):
  tokens = keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens], 250)[0]
  def predict_message(pred_text):
    encoded_text = encoded_text(text)
    pred = np.zeros((1, 250))
    pred[0] = encoded_text
    result = model.predict(pred)
    print(result[0])

  return (prediction)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])

def predict_message(pred_text):



  return (prediction)


pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
